In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf

tf.random.set_seed(0)
np.random.seed(0)

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Please specify the location of the train.csv file here:

In [3]:
folder = '/content/drive/MyDrive/Colab Notebooks/food/'

In [4]:
def load_image(img, training):
    
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.cast(img, tf.float32)
    
    img = tf.image.resize(img, (224, 224))
    
    if training:
        img = tf.image.random_flip_left_right(img)
        img = tf.image.random_flip_up_down(img)
    return img

In [5]:
def load_triplets(triplet, training):
    
    anchor = load_image(tf.io.read_file(folder + triplet[0] + '.jpg'), training)
    truthy = load_image(tf.io.read_file(folder + triplet[1] + '.jpg'), training)
    falsy = load_image(tf.io.read_file(folder + triplet[2] + '.jpg'), training)
    
    if training:
        return tf.stack([anchor, truthy, falsy], axis=0), 1
    else:
        return tf.stack([anchor, truthy, falsy], axis=0)

In [6]:
def load_dataset(triplet, training):
    
    dataset = tf.data.Dataset.from_tensor_slices(triplet)
    
    dataset = dataset.map(lambda triplet: load_triplets(triplet, training))
    return dataset

In [7]:
triplets_train = [[str(idx) for idx in line.split()] for line in open('/content/drive/MyDrive/Colab Notebooks/train_triplets.txt')]

triplets_train, triplets_val = train_test_split(triplets_train, test_size=0.2)

train_dataset = load_dataset(triplets_train, training = True).shuffle(1024, reshuffle_each_iteration=True).repeat().batch(32)
val_dataset = load_dataset(triplets_val, training = True).batch(32)

In [8]:
def create_model(freeze=True):
    
    inputs = tf.keras.Input(shape=(3, 224, 224, 3))
    
    feature_model = tf.keras.applications.MobileNetV3Large(include_top=False, input_shape=(224, 224, 3))
    feature_model.trainable = not freeze
    
    classifier = tf.keras.Sequential([
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(256),
        tf.keras.layers.Lambda(
            lambda t: tf.math.l2_normalize(t, axis=1))])
    
    anchor, truthy, falsy = inputs[:, 0, ...], inputs[:, 1, ...], inputs[:, 2, ...]

    anchor_features = classifier(feature_model(anchor))
    truthy_features = classifier(feature_model(truthy))
    falsy_features = classifier(feature_model(falsy))
    
    embeddings = tf.stack([anchor_features, truthy_features, falsy_features], axis=-1)
    
    triple_siamese = tf.keras.Model(inputs=inputs, outputs=embeddings)
    triple_siamese.summary()
    
    return triple_siamese

In [9]:
def compute_distances(embeddings):
    
    anchor, truthy, falsy = embeddings[..., 0], embeddings[..., 1], embeddings[..., 2]
    
    distance_truthy = tf.reduce_sum(tf.square(anchor - truthy), 1)
    distance_falsy = tf.reduce_sum(tf.square(anchor - falsy), 1)
    
    return distance_truthy, distance_falsy

In [10]:
def create_inference_model(model):
    
    distance_truthy, distance_falsy = compute_distances(model.output)
    
    predictions = tf.cast(tf.greater_equal(distance_falsy, distance_truthy), tf.int8)
    
    return tf.keras.Model(inputs=model.inputs, outputs=predictions)

In [11]:
def triplet_loss(_, embeddings):
    
    distance_truthy, distance_falsy = compute_distances(embeddings)
    
    return tf.reduce_mean(tf.math.softplus(distance_truthy - distance_falsy))

In [12]:
def accuracy(_, embeddings):
    
    distance_truthy, distance_falsy = compute_distances(embeddings)
    
    return tf.reduce_mean(tf.cast(tf.greater_equal(distance_falsy, distance_truthy), tf.float32))

In [13]:
model = create_model()

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                  loss=triplet_loss,
                  metrics=[accuracy])


history = model.fit(
        train_dataset,
        steps_per_epoch=int(np.ceil(len(triplets_train) / 32)),
        epochs=5,
        validation_data=val_dataset,
        validation_steps=10)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 3, 224, 224  0           []                               
                                , 3)]                                                             
                                                                                                  
 tf.__operators__.getitem (Slic  (None, 224, 224, 3)  0          ['input_1[0][0]']                
 ingOpLambda)                                                                                     
                                                                                                  
 tf.__operators__.getitem_1 (Sl  (None, 224, 224, 3)  0          ['input_1[0][0]']                
 icingOpLambda)                                                                               

In [14]:
triplets_test = [[str(idx) for idx in line.split()] for line in open('/content/drive/MyDrive/Colab Notebooks/test_triplets.txt')]

test_dataset = load_dataset(triplets_test, training = False).batch(256).prefetch(2)

In [15]:
inference_model = create_inference_model(model)

predictions = inference_model.predict(
        test_dataset,
        steps=int(np.ceil(len(triplets_test) / 256)),
        verbose=1)

233/233 [==============================] - 1245s 5s/step


In [16]:
np.savetxt('/content/drive/MyDrive/Colab Notebooks/predictions.txt', predictions, fmt='%i')